In [40]:
import os
import numpy as np
from bioio import BioImage
import bioio_ome_tiff
from tifffile import imsave, imwrite, imread
import matplotlib.pyplot as plt
import bioio_ome_tiff
from aicssegmentation.core.pre_processing_utils import suggest_normalization_param
# from bioio.writers import OmeTiffWriter

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
# from aicsimageio import AICSImage
# from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_2d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d, edge_preserving_smoothing_3d
from skimage.morphology import remove_small_objects    
from aicssegmentation.core.utils import topology_preserving_thinning



In [41]:
file_path = "/allen/aics/users/suraj.mishra/MegaSeg/Dual_Mito/data/ch1/AD00005684_20231208_X02_001.czi - AD00005684_20231208_X02_001.czi #01.tif_ch=1.tiff"
img = imread(file_path)
img.shape

(75, 624, 924)

In [42]:
# pre-processing

# parameters needed
intensity_scaling_param = [1, 15]
gaussian_smoothing_sigma = 1

# intensity normalization
struct_img = intensity_normalization(img, scaling_param=intensity_scaling_param)

# smoothing with 2d gaussian filter slice by slice 
structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)

In [46]:
# Core algorithm

# parameters
f2_param = [[1.6, 0.2]] #[1.5, 0.16]
thin_dist_preserve=3
thin_dist=1

# 2D filament filter
bw = filament_2d_wrapper(structure_img_smooth, f2_param)

# topology thinning
bw_thin = topology_preserving_thinning(bw>0, thin_dist_preserve, thin_dist)



In [47]:
# post-processing

# min area
minArea = 5

# remove small objects
seg = remove_small_objects(bw_thin>0, min_size=minArea, connectivity=1)

In [48]:
# save segmented image 
seg = seg >0
out=seg.astype(np.uint8)
out[out>0]=255

imwrite("SEG_CH1_AD00005684_20231208_X02_001.tiff", out, imagej=True)

In [49]:
# batch processing

input_folder = "/allen/aics/users/suraj.mishra/MegaSeg/Dual_Mito/data/ch1"
output_folder = "/allen/aics/users/sandra.oluoch/aics-segmentation/mitochondria/ch1_seg"

# parameters needed
intensity_scaling_param = [1, 15]
gaussian_smoothing_sigma = 1
f2_param = [[1.6, 0.2]] #[1.5, 0.16]
thin_dist_preserve=3
thin_dist=1
minArea = 5

for file_name in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file_name)
    img = imread(file_path)
    
    # pre-processing
    # intensity normalization
    struct_img = intensity_normalization(img, scaling_param=intensity_scaling_param) 
    # smoothing with 2d gaussian filter slice by slice 
    structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)
    
    # core algorithm
    # 2D filament filter
    bw = filament_2d_wrapper(structure_img_smooth, f2_param)

    # topology thinning
    bw_thin = topology_preserving_thinning(bw>0, thin_dist_preserve, thin_dist)
    
    # core algorithm
    # remove small objects
    seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1)
    
    # save segmented image 
    seg = seg >0
    out=seg.astype(np.uint8)
    out[out>0]=255

    imwrite(output_folder + "/" + "SEG_" + "{}".format(file_name) + ".tiff", out, imagej=True)
    